In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

from tabulate import tabulate


print("--")


In [ ]:

import dask.dataframe as dd
import numpy as np
import pandas as pd
   #
import json

from tabulate import tabulate

print("--")


In [ ]:

#  Load DataFrame with raw input data

l_InputFiles  = [
   "/home/jovyan/work/My_KG_NoteBooks/N4_GNN_MovieDB/02_Files/50_Keywords_00.txt",
]

df_data = dd.read_csv(
   l_InputFiles,
   delimiter  = ",",
   skiprows   = 1,                                            #  Skip the first line of each file, since it's the column headers
   dtype      = {
      "id"                        : np.dtype(str),
      "keywords"                  : np.dtype(str),            #  In the source CSV, this column was titled 'values', a bad idea
      },
   names      = [
      "id", "keywords"
      ]
   )   

df_data.compute()

print("--")


In [ ]:
# ##########################################################

In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [ ]:
#  CREATE GRAPHS

my_graph=my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)
print("")


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())


In [ ]:

l_result = my_graph.query("""

   MATCH (n) -[r]-> (m)
   RETURN n, r, m
   
   """,
   contextualize=True)

l_result.view()


In [ ]:
# ##########################################################

In [ ]:

da_data      = df_data.to_dask_array()

df_data2     = dd.io.from_dask_array(da_data,   columns=[
   ])
df_data2.compute()

df_keywords2   = dd.io.from_dask_array(da_keywords, columns=["id", "name", "LABEL"])
df_keywords2.compute()
   #
df_edges2      = dd.io.from_dask_array(da_edges,    columns=["movie_id", "keyword_id", "TYPE"])
df_edges2.compute()

#  da_movies   = []
#  da_keywords = []
#  da_edges    = []


print("--")


In [ ]:

#  Need to drop movies.genres_json because of this issue,
#
#     ValueError: Failed to convert partition to expected pyarrow schema:
#         `ArrowTypeError("Expected bytes, got a 'list' object", 'Conversion failed for column genres_json with type object')`
#     
#     Expected partition schema:
#         id: string
#         title: string
#              ...
#         vote_count: string
#         genres_json: string
#         genres_primary: string
#         genres_primary_id: string
#         LABEL: string
#     
#     Received partition schema:
#         id: string
#         title: string
#              ...
#         vote_count: string
#         genres_json: list<item: struct<id: int64, name: string>>
#           child 0, item: struct<id: int64, name: string>
#               child 0, id: int64
#               child 1, name: string
#         genres_primary: string
#         genres_primary_id: int64
#         LABEL: string


df_movies2.drop("genres_json", axis=1)
df_movies2.compute()

print("--")
    

In [ ]:

with import_data.DataFrameImporter(my_graph1) as df_importer:
    
   #  Movies
   #
   df_importer.nodes_dataframe(
      df_movies2[[
         "id", "title", "overview", "tagline", "budget", "genres", "popularity", "production_companies",
         "release_date", "revenue", "runtime", "vote_average", "vote_count", "genres_json", "genres_primary",
         "genres_primary_id", "LABEL"
      ]],
      id_column  = "id",
      id_space   = "Movies"
      )

   #  Keywords
   #
   df_importer.nodes_dataframe(
      df_keywords2[["id", "name", "LABEL"]],
      id_column  = "id",
      id_space   = "Keywords"
      )  
    
   #  Edge, RELATES_TO
   #
   df_importer.edges_dataframe(
      df_edges2[["movie_id", "keyword_id", "TYPE"]],
      source_id_space      = "Movie",
      destination_id_space = "Keywords",
      source_column        = "movie_id",
      destination_column   = "keyword_id",
      type                 = "RELATES_TO"
      )
   df_importer.edges_dataframe(
      df_edges2[["movie_id", "keyword_id", "TYPE"]],
      source_id_space      = "Keywords",
      destination_id_space = "Movies",
      source_column        = "keyword_id",
      destination_column   = "movie_id",
      type                 = "RELATES_TO"
      )

   df_importer.execute()
    
    